In [1]:
# # ! pip install transformers
# # # # ! pip install torch
# ! pip install pandas
# ! pip install tqdm
# ! pip install sklearn
# ! pip install gensim
# ! pip install nltk
# # # import sys,os
# # # os.path.dirname(sys.executable)


### Load libraries

In [2]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import itertools
# import swifter
from scipy.spatial.distance import cosine
from collections import Counter
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
import gensim.downloader
tqdm.pandas()

### Load Dataset

In [3]:
path_dir =  os.path.dirname(os.getcwd())
train_df = pd.read_csv(os.path.join(path_dir,r'data\interim\train_data.csv'))
val_df = pd.read_csv(os.path.join(path_dir,r'data\interim\val_data.csv'))
train_df.drop('Unnamed: 0',axis=1,inplace = True)
val_df.drop('Unnamed: 0',axis=1,inplace = True)
train_df = train_df.dropna()
val_df = val_df.dropna()
train_df.head(5)

,id,title,context,question,answer,answer_start,is_impossible
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269,False
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207,False
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526,False
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166,False
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276,False


In [125]:
val_df.drop_duplicates(keep = 'first', inplace = True)
val_df.head(5)

,id,title,context,question,answer,answer_start,is_impossible,answer_end
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,False,165
4,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries,94,False,117
5,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,in the 10th and 11th centuries,87,False,117
8,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"Denmark, Iceland and Norway",256,False,283
12,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,Rollo,308,False,313


#### Get answer start and end index

In [4]:
def add_end_idx(answer, start_idx , context):
#     print(answer,start_idx,context)
    end_idx = start_idx + len(answer)
    if context[start_idx:end_idx] == answer:
        return start_idx, end_idx
    else:
        for offset in [1,2]:
            if context[start__idx-offset:end_idx-offset] == answer:
                return start_idx-offset, end_idx-offset

In [5]:
train_df[['answer_start','answer_end']]= train_df[['answer','answer_start','context']]\
.progress_apply(lambda x: add_end_idx(x[0],x[1],x[2]),axis=1,result_type="expand")
val_df[['answer_start','answer_end']]= val_df[['answer','answer_start','context']]\
.progress_apply(lambda x: add_end_idx(x[0],x[1],x[2]),axis=1,result_type="expand")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 20302/20302 [00:00<00:00, 34989.40it/s]


In [6]:
train_df.head()

,id,title,context,question,answer,answer_start,is_impossible,answer_end
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269,False,286
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207,False,226
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526,False,530
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166,False,180
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276,False,286


In [7]:
train_contexts, train_questions = train_df['context'].tolist(),train_df['question'].tolist()
train_answers = [{'text': ans, 'answer_start':ans_start, 'answer_end':ans_end} 
                 for ans,ans_start,ans_end in 
                 zip(train_df['answer'].tolist(),train_df['answer_start'].tolist(),train_df['answer_end'].tolist())] 

In [126]:
val_contexts, val_questions = val_df['context'].tolist(),val_df['question'].tolist()
val_answers = [{'text': ans, 'answer_start':ans_start, 'answer_end':ans_end} 
                 for ans,ans_start,ans_end in 
                 zip(val_df['answer'].tolist(),val_df['answer_start'].tolist(),val_df['answer_end'].tolist())] 

In [127]:
val_answers[:5]

[{'text': 'France', 'answer_start': 159, 'answer_end': 165},
 {'text': '10th and 11th centuries', 'answer_start': 94, 'answer_end': 117},
 {'text': 'in the 10th and 11th centuries',
  'answer_start': 87,
  'answer_end': 117},
 {'text': 'Denmark, Iceland and Norway',
  'answer_start': 256,
  'answer_end': 283},
 {'text': 'Rollo', 'answer_start': 308, 'answer_end': 313}]

---
### Tokenize/Encode

In [10]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

C:\Users\prasr\anaconda3\envs\temp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
train_encodings = tokenizer(train_contexts, train_questions, truncation = True, padding = True)

In [128]:
val_encodings = tokenizer(val_contexts, val_questions, truncation = True, padding = True)

In [12]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [92]:
# tokenizer.decode(train_encodings['input_ids'][0])

In [14]:
def add_token_positions(encodings,answers):
    start_pos, end_pos = [], []
    for i in range(len(answers)):
        start_pos.append(encodings.char_to_token(i,answers[i]['answer_start']))
        end_pos.append(encodings.char_to_token(i,answers[i]['answer_end']))
        if start_pos[-1] is None:
            start_pos[-1] = tokenizer.model_max_length
        go_back = 1
        while end_pos[-1] is None:
            end_pos[-1] = encodings.char_to_token(i,answers[i]['answer_end']-go_back)
            go_back+=1
        
    encodings.update({
        'start_positions': start_pos,
        'end_positions': end_pos
    })

add_token_positions(train_encodings,train_answers)

In [129]:
add_token_positions(val_encodings,val_answers)

In [15]:
# train_encodings['end_positions'][:100]

In [16]:
# create a pytorch data object to load the data using pytorch dataloader later
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self,idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [17]:
train_dataset = SquadDataset(train_encodings)

In [130]:
val_dataset = SquadDataset(val_encodings)

---
#### Fine-tune Distil Bert

In [18]:
#there are different heads on top of traditional bert, we are here using QuestionAnswering head
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [19]:
from torch.utils.data import DataLoader
from transformers import AdamW
import torch

In [20]:
# torch.backends.cudnn.enabled
torch.cuda.is_available()
# torch.zeros(1).cuda()

True

In [23]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [22]:
device = torch.device('cuda') if torch.cuda.is_available() else torc.device('cpu')
model.to(device)
optim = AdamW(model.parameters(),lr=5e-5)

train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)

for epoch in range(3):
    model.train()
    # to use tdm
    loop = tqdm(train_loader)
    for batch in loop:
        # initialize gradients to zero for each batch
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        
        outputs = model(input_ids,attention_mask = attention_mask,
                       start_positions= start_positions,
                       end_positions = end_positions)
        
        loss = outputs[0]
        loss.backward()
        optim.step()
        
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())      

model_path = r'C:\Users\prasr\Documents\Northeastern\Sem2\Stanford Question Answering Dataset - PROJECT\Question-Answering-model\models\distilbert_custom'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

#### Switch out of training mode to eval mode

In [28]:
model.eval()

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [139]:
val_loader = DataLoader(val_dataset,batch_size = 8)
acc = []
predicted_answers = []
# to use tdm
loop = tqdm(val_loader)
for batch in loop:
    # stop pytorch from calculating any gradients
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)

        outputs = model(input_ids,attention_mask = attention_mask)
        start_preds = torch.argmax(outputs['start_logits'],dim=1)
        end_preds = torch.argmax(outputs['end_logits'],dim=1)
        acc.append(((start_preds == start_true).sum()/len(start_preds)).item())
        acc.append(((end_preds == end_true).sum()/len(end_preds)).item())
#         predicted_raw_answers = [ (start_idx,end_idx) for start_idx,end_idx in zip(start_preds,end_preds)]
#         predicted_answers += []
#         break       


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1299/1299 [01:30<00:00, 14.28it/s]


In [140]:
sum(acc)/len(acc)

0.5586027713625866

In [115]:
predicted_raw_answers

[(tensor(41, device='cuda:0'), tensor(42, device='cuda:0')),
 (tensor(41, device='cuda:0'), tensor(42, device='cuda:0')),
 (tensor(41, device='cuda:0'), tensor(42, device='cuda:0')),
 (tensor(41, device='cuda:0'), tensor(42, device='cuda:0')),
 (tensor(28, device='cuda:0'), tensor(31, device='cuda:0')),
 (tensor(28, device='cuda:0'), tensor(31, device='cuda:0')),
 (tensor(28, device='cuda:0'), tensor(31, device='cuda:0')),
 (tensor(28, device='cuda:0'), tensor(31, device='cuda:0'))]

In [136]:
# val_contexts,val_questions
counter = 0
for input_ids,attention_mask, context, question in zip(val_encodings['input_ids'],val_encodings['attention_mask'],val_contexts,val_questions):
    model.eval()
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    input_ids = torch.tensor(input_ids).reshape(1,-1).to(device)
    attention_mask = torch.tensor(attention_mask).reshape(1,-1).to(device)
#     print(input_ids.shape)
    output = model(input_ids,attention_mask = attention_mask)
    start_scores = output['start_logits']
    end_scores = output['end_logits']
    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores)
#     answer = ' '.join(tokens[start_idx:end_idx+1])
    answer = tokens[start_idx]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_idx + 1, end_idx + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
#     print(start_idx,end_idx)
    print(f'Context {context}')
    print(f'question {question}')
    print(f'answer {answer}')
    print('-'*30)
    if counter>5:
        break
    counter+=1

Context The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
question In what country is Normandy located?
answer france .
------------------------------
Context The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a

In [137]:
# val_contexts,val_questions
answer_list= []
for input_ids,attention_mask in tqdm(zip(val_encodings['input_ids'],val_encodings['attention_mask'])):
    model.eval()
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    input_ids = torch.tensor(input_ids).reshape(1,-1).to(device)
    attention_mask = torch.tensor(attention_mask).reshape(1,-1).to(device)
#     print(input_ids.shape)
    output = model(input_ids,attention_mask = attention_mask)
    start_scores = output['start_logits']
    end_scores = output['end_logits']
    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores)
#     answer = ' '.join(tokens[start_idx:end_idx+1])
    # Start with the first token.
    answer = tokens[start_idx]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_idx + 1, end_idx + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
#     print(start_idx,end_idx)
    answer_list.append(answer)
#     break

10388it [02:14, 77.07it/s]


In [138]:
answer_list

['france .',
 '10th and 11th centuries',
 '10th and 11th centuries',
 'denmark , iceland and norway',
 'rollo ,',
 '10th',
 '10th',
 '10th',
 'william the conqueror ,',
 'richard i',
 'catholic orthodoxy',
 'catholic orthodoxy',
 'norseman , viking " .',
 'norseman , viking " .',
 '9th century )',
 '911',
 'king charles iii of west francia and the famed viking ruler rollo , and was situated in the former frankish kingdom of neustria . the treaty offered rollo and his men',
 'river seine ,',
 'river seine ,',
 "rollo '",
 'norse',
 'north',
 'north',
 'fighting horsemen',
 'the seljuk turks .',
 'the seljuk turks .',
 'the seljuk turks .',
 '1050s .',
 '1050s .',
 '1060s ,',
 '1060s ,',
 'alexius komnenos .',
 'afranji ,',
 'oursel',
 'armenian state',
 'the raoulii were descended from an italo - norman',
 'the raoulii were descended from an italo - norman',
 'the raoulii were descended from an italo - norman',
 'robert guiscard ,',
 '108',
 '108',
 '30 , 000',
 'deabolis .',
 'deabolis

In [141]:
val_df['distil_predicted_ans'] = answer_list
val_df.head(5)

,id,title,context,question,answer,answer_start,is_impossible,answer_end,distil_predicted_ans
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,False,165,france .
4,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries,94,False,117,10th and 11th centuries
5,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,in the 10th and 11th centuries,87,False,117,10th and 11th centuries
8,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"Denmark, Iceland and Norway",256,False,283,"denmark , iceland and norway"
12,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,Rollo,308,False,313,"rollo ,"


In [147]:
distil_bert_answers = {}
for index,row in val_df.iterrows():
    distil_bert_answers[row['id']] = row['distil_predicted_ans']

In [148]:
distil_bert_answers

{'56ddde6b9a695914005b9628': 'france .',
 '56ddde6b9a695914005b9629': '10th and 11th centuries',
 '56ddde6b9a695914005b962a': 'denmark , iceland and norway',
 '56ddde6b9a695914005b962b': 'rollo ,',
 '56ddde6b9a695914005b962c': '10th',
 '56dddf4066d3e219004dad5f': 'william the conqueror ,',
 '56dddf4066d3e219004dad60': 'richard i',
 '56dddf4066d3e219004dad61': 'catholic orthodoxy',
 '56dde0379a695914005b9636': 'norseman , viking " .',
 '56dde0379a695914005b9637': '9th century )',
 '56dde0ba66d3e219004dad75': '911',
 '56dde0ba66d3e219004dad76': 'king charles iii of west francia and the famed viking ruler rollo , and was situated in the former frankish kingdom of neustria . the treaty offered rollo and his men',
 '56dde0ba66d3e219004dad77': 'river seine ,',
 '56dde1d966d3e219004dad8d': "rollo '",
 '56dde27d9a695914005b9651': 'norse',
 '56dde27d9a695914005b9652': 'north',
 '56dde2fa66d3e219004dad9b': 'fighting horsemen',
 '56de0f6a4396321400ee257f': 'the seljuk turks .',
 '56de0ffd43963214

In [150]:
import json
output_file = r'C:\Users\prasr\Documents\Northeastern\Sem2\Stanford Question Answering Dataset - PROJECT\Question-Answering-model\data\processed\distil_bert_fine_tuned.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(distil_bert_answers, f, ensure_ascii=False, indent=4)